# 오디오 자르기

In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf
import natsort

# 
base_path = "EBSi\서지나\[올림포스] 확률과 통계\\"
audio_path = base_path
save_path = base_path  + "save"
audio_list = os.listdir(audio_path)

#
save_list = natsort.natsorted(os.listdir(save_path))

In [ ]:
def trim_audio_data(audio_file, save_file, start_time=0, sec=595):
    sr = 44100
    y, sr = librosa.load(audio_file, sr=sr)
    ny = y[start_time*sr:sr*(sec + start_time)]

    sf.write(save_file + f"_{start_time}.wav", ny, sr)

def cut_audio(audio_list):
    for audio_name in audio_list:
        if audio_name.find('mp3') != -1:
            audio_file = audio_path + audio_name
            save_file = save_path + "\\" + audio_name[:-4]

            f = sf.SoundFile(audio_file)
            f_sec = f.frames // f.samplerate
            print(audio_file, " seconds, ", f_sec)

            sec = 595
            for i in range(0,f_sec,sec):
                trim_audio_data(audio_file, save_file, i, sec)

In [ ]:
cut_audio(audio_list)

# 잡음제거

In [ ]:
# 잡음 제거 함수
def noise_reduction(save_list):
    for save_name in save_list:
        # 파일 경로 결합 및 명령어 실행
        command = f'spleeter separate -o "{base_path}output" "{save_path}\{save_name}"'
        !{command}

In [ ]:
noise_reduction(save_list)

# 오디오 합치기

In [ ]:
from pydub import AudioSegment

def combine_audio(save_list, base_path):
    output_folder = f'{base_path}output'  # output 폴더 경로
    
    for audio_name in save_list:
        if audio_name.find('wav') != -1:
            audio_file = audio_name.split('_')[0]
            matching_files = [file for file in save_list if audio_file in file]

            # 오디오 파일을 순서대로 합치기
            combined_audio = AudioSegment.empty()
            for file in matching_files:
                audio = AudioSegment.from_file(output_folder + '\\' + file[:-4] + '\\vocals.wav')
                combined_audio += audio

            # 합쳐진 오디오 파일 저장
            combined_audio.export(f"{base_path}combined\\{audio_file}_combined_audio.wav", format="wav")

In [ ]:
combine_audio(save_list, base_path)